**Description**: This Project takes a Youtube video and translates it to a selected regional language.

Steps
1. Download the Youtube video and save locally.
2. Trim the video for silence.
3. Get the audio from the video.
4. Split the audio into senetnces of 1 minute duration.
5. Convert the speech into english transcription for each file.
6. Remove the cuss words from the movie.
7. Translate the english transcription into local language.
8. Evaluate and re-experiment

TODO: Try Google Translate
https://codelabs.developers.google.com/codelabs/cloud-translation-python3#4

Designing the project:
1. Make sure that the code is efficient for API calls
2. Failures should be gracefully handled.
3. Cost effectiveness
4. Logging
5. Quality of the outcome.
6. Maintainability

In [1]:
from getpass import getpass
openaikey = getpass('##')

In [2]:
chatgpt_url = "https://api.openai.com/v1/chat/completions"
chatgpt_headers = {
    "content-type": "application/json",
    "Authorization":"Bearer {}".format(openaikey)}

In [3]:
import openai
openai.api_key = openaikey

In [4]:
#import openai
import os
import urllib
from IPython.display import Audio
from pathlib import Path
from pydub import AudioSegment

C:\Users\tanay\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [5]:
# from pytube import YouTube
#
# # Replace 'VIDEO_URL' with the URL of the YouTube video you want to download
# video_url = 'https://www.youtube.com/watch?v=Tg7jAnS78JA'
#
# # Create a YouTube object
# yt = YouTube(video_url)
#
# # Choose the stream with the desired quality and resolution
# # You can loop through available streams and select the one you want
# stream = yt.streams.get_lowest_resolution()
#
# # Specify the download location (optional)
# # If not specified, it will be downloaded to the current working directory
# download_location = 'D:\\AIML\\Projects\\'
#
# # Download the video
# stream.download(output_path=download_location)
#
# print(f"Video '{yt.title}' downloaded successfully!")
#
#
#
#
#


In [6]:
# Function to detect leading silence
# Returns the number of milliseconds until the first sound (chunk averaging more than X decibels)
def milliseconds_until_sound(sound, silence_threshold_in_decibels=-20.0, chunk_size=10):
    trim_ms = 0  # ms

    assert chunk_size > 0  # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold_in_decibels and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

In [20]:
def trim_start(filepath):
    path = Path(filepath)
    print(path)
    directory = path.parent
    print(directory)
    filename = path.name
    print(filename)
    audio = AudioSegment.from_file(filepath, format="mp4")
    # audio = AudioSegment.from_mp3(filename)
    print("Audio")
    start_trim = milliseconds_until_sound(audio)
    trimmed = audio[start_trim:]
    new_filename = directory / f"trimmed_{filename}"
    trimmed.export(new_filename, format="mp4")
    return trimmed, new_filename

In [21]:
file_path="D:\\PyTorch\\Projects\\The_deal.mp4"
trimmed_audio = trim_start(file_path)

D:\PyTorch\Projects\The_deal.mp4
D:\PyTorch\Projects
The_deal.mp4


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [14]:
trimmed_audio, trimmed_filename = trim_start(file_path)

FileNotFoundError: [WinError 2] The system cannot find the file specified

# Option-1: Calling Transcription on whole speech

   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ------------------------------------- -- 71.7/76.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.53.0
    Uninstalling openai-1.53.0:
      Successfully uninstalled openai-1.53.0


In [30]:
# Get the Transcription for the Audio
# with open(trimmed_filename, 'rb') as audio_data:
with open("D:\\PyTorch\\Projects\\The_deal.mp4", 'rb') as audio_data:
    total_transcript = (openai.Audio.transcribe("whisper-2", audio_data))["text"]
total_transcript

APIRemovedInV1: 

You tried to access openai.Audio, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [16]:
def Moderation_assistant(english_transcript):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        top_p=0.7,
        messages=[
            {
                "role": "system",
                "content": "You are moderation assistant. Add write punctuation and Replace all the adult content words, cuss words, and abusive words with *** "
            },
            {
                "role": "user",
                "content": english_transcript
            }
        ]
    )
    return response

In [17]:
def Translation_assistant(english_transcript, translation_language):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0.5,
        top_p=1,
        messages=[
            {
                "role": "system",
                "content": "Translate the english sentencences into "+translation_language+". Use the following examples for translation. Do you mind passing the sauce? should be translated to సాస్ పాస్ చేయడం మీకు అభ్యంతరమా? ignore the repetition of the characters or words or sentences in the end. Use the following example for translation."
            },
            {
                "role": "user",
                "content": english_transcript
            }
        ]
    )
    return response



In [18]:
moderated_transcript = (Moderation_assistant(total_transcript)) ['choices'][0]['message']['content']
moderated_transcript


NameError: name 'total_transcript' is not defined

In [19]:
telugu_transcript = Translation_assistant(moderated_transcript,"English")

#print(ascii_transcriptions)
telugu_transcript['choices'][0]['message']['content']

NameError: name 'moderated_transcript' is not defined

# Option -2 : Splitting into chunks and running the transcription

In [ ]:
# Split the audio into audio segments of 1 min duration

trimmed_audio = AudioSegment.from_file(trimmed_filename)  # Load the trimmed audio file

one_minute = 1 * 60 * 1000  # Duration for each segment (in milliseconds)

start_time = 0  # Start time for the first segment

#no_of_files = 0  # Index for naming the segmented files

output_dir_trimmed = "TrimmedMovieDirectory"  # Output directory for the segmented files

if not os.path.isdir(output_dir_trimmed):  # Create the output directory if it does not exist
    os.makedirs(output_dir_trimmed)
i=0
while start_time < len(trimmed_audio):  # Loop over the trimmed audio file
    segment = trimmed_audio[start_time:start_time + one_minute]  # Extract a segment
    segment.export(os.path.join(output_dir_trimmed, f"trimmed_{i:02d}.mp4"), format="mp4")  # Save the segment
    start_time += one_minute # Update the start time for the next segment
    i=i+1

    #no_of_files += 1  # Increment the index for naming the next file

In [ ]:
# Sort the audio files
audio_files = sorted(
    (f for f in os.listdir(output_dir_trimmed) if f.endswith(".mp4")),
    key=lambda f: int(''.join(filter(str.isdigit, f)))
)

In [ ]:
import time
sleep_time=40
def transcribe_audio(file,output_dir):
    audio_path = os.path.join(output_dir, file)
    with open(audio_path, 'rb') as audio_data:
        transcription = openai.Audio.transcribe("whisper-1", audio_data)
        time.sleep(sleep_time)
        return transcription['text']

In [ ]:
# Print Transcriptions of all the sentences
transcriptions = [transcribe_audio(file, output_dir_trimmed) for file in audio_files]
print(transcriptions)


["Here we are. You know, usually your mother and I would reserve a bottle like this for special occasions. But then I thought, you know what? For our Lauren, I'm sure we can make an exception. Angela. Hmm? We can make an exception for Angela. Dinner's ready. Ah, smells lovely Sue. Let's hope it tastes as good as it smells. As promised, your favourite. Tuck in everyone. Don't need to tell me twice. Mmm.", "Yeah, yeah, I'm good. You haven't touched your food. Usually you'd have that chicken egg before it had even touched the plate. It used to be a nightmare to get you to eat anything else. That's right. If you'd had your way growing up, we'd have had roast chicken. I'm a vegetarian. Since when? Since forever. You... You know this. How can you be a vegetarian if you love roast chicken? I don't. Are you sure? Yeah, pretty sure. Well, I'm pretty sure I know what my daughter's favourite meal is. It's not my... It's not my favourite meal. That's okay, I'll just, um... I'll just eat around the

In [ ]:
print(transcriptions[0])

Here we are. You know, usually your mother and I would reserve a bottle like this for special occasions. But then I thought, you know what? For our Lauren, I'm sure we can make an exception. Angela. Hmm? We can make an exception for Angela. Dinner's ready. Ah, smells lovely Sue. Let's hope it tastes as good as it smells. As promised, your favourite. Tuck in everyone. Don't need to tell me twice. Mmm.


In [ ]:
import time
sleep_time=40
total_moderated_transcript=""
total_translated_transcript=""
for index in range(0,i):
    moderated_transcript = (Moderation_assistant(transcriptions[index])) ['choices'][0]['message']['content']
    print(moderated_transcript)
    telugu_transcript = (Translation_assistant(moderated_transcript ,"Telugu"))['choices'][0]['message']['content']
    print(telugu_transcript)
    total_moderated_transcript += moderated_transcript
    total_translated_transcript +=telugu_transcript
    time.sleep(sleep_time)


Here we are. You know, usually your mother and I would reserve a bottle like this for special occasions. But then I thought, you know what? For our Lauren, I'm sure we can make an exception. Angela. Hmm? We can make an exception for Angela. Dinner's ready. Ah, smells lovely, Sue. Let's hope it tastes as good as it smells. As promised, your favourite. Tuck in, everyone. Don't need to tell me twice. Mmm.
ఇక్కడ మేము. మీరు మీ తల్లి మరియు నేను సాధారణంగా ఈ రకమైన ఒక బాటిల్ను ప్రత్యేక సందర్భాలకు కనుగొనటం ఉండింది. కానీ తరువాత, నాకు ఆలోచన వచ్చింది, మీరు ఏమీ చేసుకోవడం తెలుసు. మా లారెన్ కోసం, నాకు ఖచ్చితంగా ఒక అనుకూలం చేస్తే బాగా ఉంటుందని నేను అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే అనుకూలం చేస్తే 

In [ ]:
moderated_file = open(download_location+"moderatedfile.txt", "w")
moderated_file.write(total_moderated_transcript)
moderated_file.close()
translated_file = open(download_location+"tel_translation.txt", "w")
translated_file.write(total_translated_transcript)
translated_file.close()

Measure E2E quality;
Measuring Quality:
1. Spelling Errors
2. English words
3. Repetetion
4. Un-recognizable characters
5. Grammatical errors
6. Coherence of sentence
7. Word-by-word translation

Next steps:
1. Build a REST API which takes Video URL, language and sends the response. Limit the input tokens.
2. Build a web app which accepts video /URL, language and displays the content into the language.
3. Take the input stream frame by frame and overlay it on the video and reproduce it.
4. Convert the text to audio and with human face and reproduce it.
5.Should support all video formats.
6. Create a summary and sentiment of the movie.